# Example Notebook

This is an example notebook to demonstrate how your learning journal works.


## Code Examples

You can add code and it will be executed when the site is built:


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create some example data
x = np.linspace(0, 10, 100)
y = np.sin(x)

plt.figure(figsize=(10, 4))
plt.plot(x, y)
plt.title('Example Plot')
plt.xlabel('x')
plt.ylabel('sin(x)')
plt.grid(True)
plt.show()


## Notes

Add your learning notes here. You can use:
- Markdown formatting
- Code cells
- Math equations: $E = mc^2$
- Images
- And much more!
